# LoC image analysis

This notebook is designed to take the raw output of a LoC data set, monolayer or bilayer, and:
 (trying to check if dask multichan works)

5. Localise and measure properties
6. Unite the localisations over the z-range
7. Save out the Z-tracks
8. Extract the maximum intensity from each cell

In [1]:
import os
import glob
from octopusheavy import DaskOctopusHeavyLoader
from skimage.io import imshow,  imsave, imread
import napari
import btrack
from tqdm.auto import tqdm
import dask.array as da

# Loading images

Define root path and individual experiment IDs

In [2]:
root_path = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy'

In [3]:
expt_IDs = ['co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/images/',
            'co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-VWF-iCAM1-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_70_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_150_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_262_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-CAV1-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_130_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-CAV1-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_253_/images/'
           ]

## Check to see what channels each expt has

In [4]:
for i, expt in enumerate(expt_IDs):
    images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
    if 'MASK1' in [channel.name for channel in images.channels]:
        images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
        print(i, images.channels)

0 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
1 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
2 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
3 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
4 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
5 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]


In [5]:
images['CH1']

dask.array<stack, shape=(81, 2304, 2304), dtype=uint16, chunksize=(1, 2304, 2304), chunktype=numpy.ndarray>

In [31]:
tracks[30]

,ID,t,x,y,z,parent,root,state,generation,dummy,axis_major_length,mean_intensity-0,mean_intensity-3,mean_intensity-1,mean_intensity-2,area,axis_minor_length,orientation,eccentricity
0,80,0,1850.238806,998.253731,0.000000,80,80,5,0,False,14.335579,113.223881,137.119403,123.149254,168.238806,67.0,9.301548,-1.457316,0.760922
1,80,1,1850.238806,998.253731,0.000000,80,80,5,0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,80,2,1827.879518,1010.843373,0.000000,80,80,5,0,False,12.046309,114.674699,142.722892,127.108434,177.927711,83.0,9.846851,-0.748999,0.576047
3,80,3,1825.228571,977.171429,0.000000,80,80,5,0,False,17.604173,110.142857,133.142857,111.021429,116.878571,140.0,11.378606,0.060647,0.763034
4,80,4,1825.228571,977.171429,0.000000,80,80,5,0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,80,5,1825.228571,977.171429,0.000000,80,80,5,0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,80,6,1825.686957,976.060870,0.000000,80,80,5,0,False,15.015709,110.939130,133.573913,111.730435,120.469565,115.0,10.462208,0.147468,0.717313
7,80,7,1823.177419,974.935484,0.000000,80,80,5,0,False,13.693241,109.322581,133.580645,114.838710,121.790323,62.0,6.588535,-0.745334,0.876637
8,80,8,1822.317073,973.926829,0.000000,80,80,5,0,False,13.248906,113.780488,134.317073,114.621951,122.207317,82.0,8.213110,-0.265188,0.784674
9,80,9,1821.264000,967.480000,0.000000,80,80,5,0,False,18.917160,120.592000,130.248000,115.576000,120.928000,125.0,8.524525,-0.640915,0.892714


# It won't let me save tracks to the server? But will let me save locally

In [35]:
tracks_fn = 'co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.hdf5'

In [38]:
with btrack.dataio.HDF5FileHandler(tracks_fn.replace('hdf5','csv'), "w", obj_type="obj_type_1") as hdf:
    hdf.write_tracks(tracks)

[INFO][2022/12/02 02:56:41 pm] Opening HDF file: co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.csv...
02-Dec-22 14:56:41 - btrack.dataio - INFO     - Opening HDF file: co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.csv...
[INFO][2022/12/02 02:56:42 pm] Writing objects/obj_type_1
02-Dec-22 14:56:42 - btrack.dataio - INFO     - Writing objects/obj_type_1
[INFO][2022/12/02 02:56:42 pm] Writing labels/obj_type_1
02-Dec-22 14:56:42 - btrack.dataio - INFO     - Writing labels/obj_type_1
[INFO][2022/12/02 02:56:42 pm] Loading objects/obj_type_1 (59338, 5) (59338 filtered: None)
02-Dec-22 14:56:42 - btrack.dataio - INFO     - Loading objects/obj_type_1 (59338, 5) (59338 filtered: None)
[INFO][2022/12/02 02:56:42 pm] Writing properties/obj_type_1/axis_major_length (

# Hacky fix for JAKSON

In [23]:
tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy,axis_major_length,mean_intensity-0,mean_intensity-3,mean_intensity-1,mean_intensity-2,area,axis_minor_length,orientation,eccentricity
0,93,0,1128.883333,3.766667,0.0,93,93,5,0,False,"(1,) array","(1,) array","(1,) array","(1,) array","(1,) array","(1,) array","(1,) array","(1,) array","(1,) array"


In [24]:
data, new_properties, new_graph = btrack.utils.tracks_to_napari(tracks, ndim=2)

In [16]:
root_path = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy'
expt_IDs = ['co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/images/',
            'co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-VWF-iCAM1-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_70_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_150_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_262_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-CAV1-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_130_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-CAV1-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_253_/images/'
           ]

In [17]:
for i, expt in enumerate(expt_IDs):
    images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
    if 'MASK1' in [channel.name for channel in images.channels]:
        images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
        print(i, images.channels)
        dirname = os.path.dirname(images.files('CH1')[0]).replace('images','')
        tracks_fn = os.path.join(dirname, 'tracks.h5')
        tracks_fn = tracks_fn.split('Light microscopy/')[-1].replace('/', '_')
#         tracks_fn = os.path.join(root_path, expt.replace('images/', 'tracks.hdf5'))
        print('Tracks exist:', os.path.exists(tracks_fn))

0 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: True
1 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: True
2 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: True
3 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: True
4 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: True
5 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: True


In [18]:
expt = expt_IDs[0]
images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)

In [19]:
import enum
@enum.unique
class Channels(enum.Enum): ### this is unique to the above acquisition (20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_113.vsi - 005 CF405, CF488, CF561, CF638.tif)
    DAPI = 1
    SPC = 2
    PDPN = 3
    ZO1 = 4

In [20]:
output_dir = f'/home/dayn/data/lung_on_chip/pos_neg_dist/{expt}jakson'
if not os.path.exists(output_dir):
    os.makedirs(output_dir,)

In [26]:
v = napari.Viewer()

v.add_image(images['CH1'], name = Channels.DAPI.name, blending = 'additive', colormap='blue', visible = False)
v.add_image(images['CH2'], name = Channels.SPC.name, blending = 'additive', colormap='green')
v.add_image(images['CH3'], name = Channels.PDPN.name, blending = 'additive', colormap='red')
v.add_image(images['CH4'], name = Channels.ZO1.name, blending = 'additive', colormap='gray', visible = False)
v.add_labels(images['MASK1'], visible=True)
tracker_layer = v.add_tracks(data, tail_length=0, visible = False)

@v.bind_key('t')
def SPC_extract_positive_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "SPC_positive.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "SPC_positive_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('b')
def SPC_extract_negative_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "SPC_negative.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "SPC_negative_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")

@v.bind_key('p')
def NKX21_extract_positive_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "NKX21_positive.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "NKX21_positive_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('n')
def NKX21_extract_negative_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "NKX21_negative.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "NKX21_negative_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('q')
def PDPN_extract_positive_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-2'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "PDPN_positive.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "PDPN_positive_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('c')
def PDPN_extract_negative_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-2'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "PDPN_negative.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "PDPN_negative_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encounte

Looks like there was no identifiable track there


Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/_qt/widgets/qt_viewer_dock_widget.py", line 230, in keyPressEvent
    return self._ref_qt_viewer().keyPressEvent(event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/_qt/qt_viewer.py", line 1123, in keyPressEvent
    self.canvas._backend._keyEvent(self.canvas.events.key_press, event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 622, in _keyEvent
    func(native=ev, key=key, text=str(ev.text()), modifiers=mod)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/visp

Looks like there was no identifiable track there


Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 561, in keyPressEvent
    self._keyEvent(self._vispy_canvas.events.key_press, ev)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 622, in _keyEvent
    func(native=ev, key=key, text=str(ev.text()), modifiers=mod)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/key_bindings.py", line 545, in on_key_press
    self.press_key(

...

Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 561, in keyPressEvent
    self._keyEvent(self._vispy_canvas.events.key_press, ev)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 622, in _keyEvent
    func(native=ev, key=key, text=str(ev.text()), modifiers=mod)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/key_bindings.py", line 545, in on_key_press
    self.press_key(

Looks like there was no identifiable track there


Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 561, in keyPressEvent
    self._keyEvent(self._vispy_canvas.events.key_press, ev)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 622, in _keyEvent
    func(native=ev, key=key, text=str(ev.text()), modifiers=mod)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/key_bindings.py", line 545, in on_key_press
    self.press_key(

Looks like there was no identifiable track there


Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 561, in keyPressEvent
    self._keyEvent(self._vispy_canvas.events.key_press, ev)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 622, in _keyEvent
    func(native=ev, key=key, text=str(ev.text()), modifiers=mod)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/key_bindings.py", line 545, in on_key_press
    self.press_key(

In [39]:
output_dir = f'/home/dayn/data/lung_on_chip/pos_neg_dist/{expt}gabriel'
if not os.path.exists(output_dir):
    os.makedirs(output_dir,)

In [41]:
v = napari.Viewer()

v.add_image(images['CH1'], name = Channels.DAPI.name, blending = 'additive', colormap='blue', visible = False)
v.add_image(images['CH2'], name = Channels.SPC.name, blending = 'additive', colormap='green')
v.add_image(images['CH3'], name = Channels.PDPN.name, blending = 'additive', colormap='red')
v.add_image(images['CH4'], name = Channels.ZO1.name, blending = 'additive', colormap='gray', visible = False)
v.add_labels(images['MASK1'], visible=True)
tracker_layer = v.add_tracks(data, tail_length=0, visible = False)

@v.bind_key('t')
def SPC_extract_positive_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "SPC_positive.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "SPC_positive_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('b')
def SPC_extract_negative_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "SPC_negative.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "SPC_negative_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")

@v.bind_key('p')
def NKX21_extract_positive_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "NKX21_positive.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "NKX21_positive_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('n')
def NKX21_extract_negative_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "NKX21_negative.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "NKX21_negative_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('q')
def PDPN_extract_positive_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-2'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "PDPN_positive.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "PDPN_positive_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('c')
def PDPN_extract_negative_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-2'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "PDPN_negative.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "PDPN_negative_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encounte

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in 

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in 

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in 

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in 

...

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in 

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in 

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in 

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: invalid value encountered in true_divide
  alpha = (self.head_length + self.current_time - times) / (
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:312: RuntimeWarning: divide by zero encountered in 

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_animation/_qt/keyframeslist_widget.py:156: FutureWarning: The `as_dict` kwarg default to False` since Napari 0.4.17, and will become a mandatory parameter in the future.
  self.setStyleSheet(template(qss_template, **get_theme(theme_name)))
Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/_qt/menus/plugins_menu.py", line 99, in _add_toggle_widget
    self._win.add_plugin_dock_widget(*key)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/_qt/qt_main_window.py", line 717, in add_plugin_dock_widget
    wdg = _instantiate_dock_widget(
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/_qt/qt_main_window.py", line 1365, in _instantiate_dock_widget
    return wdg_cls(**kwargs)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_animation/_qt/animation_widget.py", line 44, in __init__
    self._i

## Checking tracks prior to plotting single cell info

In [ ]:
import napari

In [ ]:
viewer = napari.Viewer()

viewer.add_image(ch0, colormap='blue', blending = 'additive')
viewer.add_image(ch3, colormap= 'gray', blending = 'additive')
viewer.add_image(ch1, colormap='red', blending = 'additive')
viewer.add_image(ch2, colormap= 'green', blending = 'additive')

viewer.add_labels(mask_stack)

viewer.add_tracks(ch1_data)
viewer.add_tracks(ch2_data)
